# Beating and the Critical Band, Combination Tones

In this lab we explore the _critical band_ by generating _primary_ beatings between pure signals having frequencies $f_1$ and $f_2$, where $f_1 - f_2$ is in the order of a few Hz.

_Secondary_ beatings may also be perceived when $f_1 \simeq k f_2$ within few Hz, for $k = n/m$ and $n,m$ small integers. Here, higher harmonics play an important role.

Finally, _Combination Tones_ can be heard due to the non-linearity of the hear apparatus, and correspond to frequencies $f_2 - f_1$ (the _Tartini third sound_) and $2 f_1 - f_2$. To explore those, we add a sound with frequency $f_c$ within few Hz of those differences.

In [15]:
import numpy as np
import scipy.signal as ss
import matplotlib.pyplot as plt
from math import log10, log
from ipywidgets import interact
import ipywidgets as widgets
import librosa    # for the fast spectrogram
import librosa.display
import simpleaudio as sa
from scipy.io.wavfile import write as wavwrite

fs = 24000
plt.rc('figure', figsize=(20, 10))
spectrogram=True

In [16]:
def plottime(s, duration):
    # plot the time series of the signal s
    plt.subplots(figsize=(25, 5)) 
    ax = plt.subplot(1, 1, 1)
    plt.plot(np.real(s))
    plt.xlim(0)
    ax.set_xlabel('t [ms]')
    maxx = int(duration*10 + 1)*100
    ax.set_xticks(np.arange(0, maxx*fs/1000, maxx*fs/10000, dtype=int))
    ax.set_xticklabels(np.arange(0, maxx, maxx/10, dtype=int))
    plt.grid(which='major')
    plt.title("Wave packet")

In [19]:
@interact(f1=widgets.IntSlider(min=50, max=2000, value=262, continuous_update=False),
          #f2=widgets.BoundedFloatText(min=50, max=2000, value=393, continuous_update=False),
          f2suf1_m=widgets.BoundedFloatText(min=1, max=1000, value=5, continuous_update=False),
          f2suf1_n=widgets.BoundedFloatText(min=1, max=1000, value=3, continuous_update=False),
          harmonics=widgets.IntSlider(min=1, max=10, value=1, continuous_update=False),
          comb_tone=widgets.RadioButtons(options=['None', 'Common Bass', 'Tartini (f2 - f1)', '2f1 - f2'],
                                         value='None', continuous_update=False),
          mistune=widgets.IntSlider(min=0, max=40, value=0, continuous_update=False),
          duration=widgets.FloatSlider(min=0.01, max=6, value=3, step=0.01, continuous_update=False),
          play=widgets.Checkbox(description='play', value=True),
          save=widgets.Checkbox(description='save', value=False),
         )
def playwavelets(f1, f2suf1_m, f2suf1_n, harmonics, comb_tone, mistune, duration, play, save):
    t = np.linspace(0, duration, int(duration * fs), False)
    f2 = f1 * f2suf1_m * 1.0 / f2suf1_n
    print('f2 = %.2f Hz' % f2)

    # generate the signal
    s = np.sin(2 * np.pi * f1 * t) + np.sin(2 * np.pi * f2 * t)

    # add the harmonics with given weights if requested
    if harmonics > 1:
        for h in range(harmonics-1):   # h = 0..harmonics-2
            s += np.sin(2 * np.pi * f1 * (h+2) * t) * 1/(h+2) + np.sin(2 * np.pi * f2 * (h+2) * t) * 1/(h+2)

    # add the combination tone if requested, with a mistuning
    if comb_tone != 'None':
        fc = f2 - f1 if comb_tone == 'Tartini (f2 - f1)' else (
             2*f1 - f2 if comb_tone == '2f1 - f2' else
             f1 / f2suf1_m)  # == f2 / f2suf1_n
        if fc < 1:
            fc = 0
        print('Comb. tone: %.2f Hz' % fc)
        if fc > 0:
            s += 2 * np.sin(2 * np.pi * (fc + mistune) * t)

    #if f1 == f2:
    #    f = np.full(int(duration * fs), f1)
    #else:
    #    #f = np.arange(f1, f2, (f2-f1) * 1.0 / int(duration * fs))    # linear chirp
    #    f = f1 * np.exp(t * log(f2/f1) / duration)    # exponential chirp, "linear perception"
    #s = np.sin(2 * np.pi * f * t)

    # apply a hanning window at the ramp up and ramp down of the signal
    w = np.hanning(s.size * 0.1)
    for i in range(int(w.size/2)):
       s[i] *= w[i]
       s[s.size-int(w.size/2)+i] *= w[int(w.size/2)+i]

    plottime(s, duration)
    if spectrogram:
        # play with hop_length and nfft values
        hop_length = 64
        n_fft = 2048
        bins_per_octave = 200
        fmin = 2
        fmax = 1024

        fig, ax = plt.subplots()
        #D = librosa.amplitude_to_db(np.abs(librosa.stft(s, hop_length=hop_length)), ref=np.max)
        D = np.abs(librosa.stft(s, hop_length=hop_length))
        img = librosa.display.specshow(D, y_axis='log', sr=fs,
                                       hop_length=hop_length, x_axis='time', ax=ax, cmap='jet',
                                       bins_per_octave=bins_per_octave, auto_aspect=False)
        ax.set_ylim([fmin, fmax])
        fig.colorbar(img, ax=ax, format="%+2.f")

    # play it
    playable = s * (2**15 - 1) / np.max(np.abs(s))
    if play:
        # stop any ongoing play
        sa.stop_all()
        # convert to 16-bit data and play
        sa.play_buffer(playable.astype(np.int16), 1, 2, fs)        

    if save:
        wavwrite('%d_su_%d.wav' % (f2suf1_m, f2suf1_n), fs, playable.astype(np.int16))

interactive(children=(IntSlider(value=262, continuous_update=False, description='f1', max=2000, min=50), Bound…

## Beatings with a chirp from f to 2f

We now analyze a chirp and how it beats with a constant f. The chirp has a frequency evolution going like a sigmoid from f to 2f, to enhance the primary beatings at the beginning and the secondary beatings of the mistuned octave at the end.

In [9]:
@interact(f1=widgets.IntSlider(min=50, max=1000, value=256, continuous_update=False),
          duration=widgets.FloatSlider(min=5, max=60, value=41, step=1, continuous_update=False),
          play=widgets.Checkbox(description='play', value=True),
          save=widgets.Checkbox(description='save', value=False))
def playchirp(f1, duration, play, save):
    t = np.linspace(0, duration, num=int(duration * fs), endpoint=True)

    # generate the signal
    s = np.sin(2 * np.pi * f1 * t) + \
        ss.chirp(t, f0=0.98*f1, f1=2.04*f1, t1=duration, method='log')
    #s = np.sin(2 * np.pi * f1 * t) + \
    #    np.concatenate(
    #        (ss.chirp(np.linspace(0, 2, num=2*fs, endpoint=False),
    #                  f0=f1, f1=1.08*f1, t1=2, method='linear'),
    #         ss.chirp(np.linspace(2, duration, num=int((duration-2)*fs), endpoint=True),
    #                  f0=1.08*f1, f1=2.05*f1, t1=duration, method='log', phi=180)))
    #         ss.chirp(np.linspace(duration-2, duration, num=2*fs, endpoint=True),
    #                  f0=1.5*f1, f1=2*f1, t1=duration, method='quadratic', vertex_zero=False)))

    # apply a hanning window at the ramp up and ramp down of the signal
    w = np.hanning(s.size * 0.01)
    for i in range(int(w.size/2)):
       s[i] *= w[i]
       s[s.size-int(w.size/2)+i] *= w[int(w.size/2)+i]

    plottime(s, duration)
    if spectrogram:
        # play with hop_length and nfft values
        hop_length = 64
        n_fft = 1024
        bins_per_octave = 256
        fmin = f1*0.9
        fmax = f1*2.2

        fig, ax = plt.subplots()
        #D = librosa.amplitude_to_db(np.abs(librosa.stft(s, hop_length=hop_length)), ref=np.max)
        D = np.abs(librosa.stft(s, hop_length=hop_length))
        img = librosa.display.specshow(D, y_axis='log', sr=fs,
                                       hop_length=hop_length, x_axis='time', ax=ax, cmap='jet',
                                       bins_per_octave=bins_per_octave, auto_aspect=False)
        ax.set_ylim([fmin, fmax])
        ax.set_yticks([256, 290, 341.3, 384, 512])
        ax.grid(True)
        fig.colorbar(img, ax=ax, format="%+2.f")

    # play it
    playable = s * (2**15 - 1) / np.max(np.abs(s))
    if play:
        # stop any ongoing play
        sa.stop_all()
        # convert to 16-bit data and play
        sa.play_buffer(playable.astype(np.int16), 1, 2, fs)
    
    if save:
        wavwrite('chirp.wav', fs, playable.astype(np.int16))

interactive(children=(IntSlider(value=256, continuous_update=False, description='f1', max=1000, min=50), Float…